<a href="https://colab.research.google.com/github/czhang2718/LLGM/blob/main/notebooks/base_deepseek_setup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install transformers
!pip3 install torch
!pip3 install huggingface_hub
!pip3 install flash_attn
!pip3 install accelerate
!pip3 install datasets
!pip3 install evaluate
!pip3 install numpy

DEPRECATION: pytorch-lightning 1.6.5 has a non-standard dependency specifier torch>=1.8.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


DEPRECATION: pytorch-lightning 1.6.5 has a non-standard dependency specifier torch>=1.8.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


DEPRECATION: pytorch-lightning 1.6.5 has a non-standard dependency specifier torch>=1.8.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


^C
     ---------------------------------------- 0.0/2.6 MB ? eta -:--:--
     -------- ------------------------------- 0.5/2.6 MB 16.5 MB/s eta 0:00:01
     ----------------------- ---------------- 1.5/2.6 MB 16.1 MB/s eta 0:00:01
     ------------------------------------- -- 2.4/2.6 MB 15.5 MB/s eta 0:00:01
     ---------------------------------------- 2.6/2.6 MB 13.8 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'


  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [22 lines of output]
      fatal: not a git repository (or any of the parent directories): .git
      
      
      torch.__version__  = 2.2.0+cpu
      
      
      C:\Users\czhan\AppData\Local\Temp\pip-install-sollwp6n\flash-attn_3011c37a9994415bb054946d18f51ebb\setup.py:78: UserWarning: flash_attn was requested, but nvcc was not found.  Are you sure your environment has nvcc available?  If you're installing within a container from https://hub.docker.com/r/pytorch/pytorch, only images whose names contain 'devel' will provide nvcc.
        warnings.warn(
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 34, in <module>
        File "C:\Users\czhan\AppData\Local\Temp\pip-install-sollwp6n\flash-attn_3011c37a9994415bb054946d18f51ebb\setup.py", line 134, in <module>
          CUDAExtension(


In [1]:
model_name = "deepseek-ai/DeepSeek-Coder-V2-Lite-Instruct"
cache_name = ".depsek"
dataset_name = "AnkitAI/CategorizedTextReviews"

In [7]:
from transformers import AutoModel, AutoConfig, AutoTokenizer, TrainingArguments, Trainer
from huggingface_hub import snapshot_download, notebook_login
import torch
from datasets import load_dataset
import numpy as np
import evaluate

ModuleNotFoundError: No module named 'datasets'

In [4]:
notebook_login()

In [ ]:
snapshot_dir = snapshot_download(repo_id=model_name, cache_dir=cache_name)
print(snapshot_dir)

In [ ]:
model_base = AutoModel.from_pretrained(
    model_name,
    cache_dir=cache_name,
    trust_remote_code=True,
    local_files_only=True,
    low_cpu_mem_usage=True,
    torch_dtype=torch.bfloat16
)

import sys
sys.path.append(".depsek/models--deepseek-ai--DeepSeek-Coder-V2-Lite-Instruct/snapshots/")
!touch {snapshot_dir}/__init__.py
!ls {snapshot_dir}

In [ ]:
from e434a23f91ba5b4923cf6c9d9a238eb4a08e3a11 import modeling_deepseek

In [ ]:
config = AutoConfig.from_pretrained(
    model_name,
    cache_dir=cache_name,
    trust_remote_code=True,
    local_files_only=True,
    low_cpu_mem_usage=True,
    torch_dtype=torch.bfloat16
)
config.hidden_size = 100

In [ ]:
model_classify = modeling_deepseek.DeepseekV2ForSequenceClassification(config)
model_classify.model = model_base

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    cache_dir=cache_name,
    trust_remote_code=True,
    local_files_only=True,
    low_cpu_mem_usage=True,
    torch_dtype=torch.bfloat16
)

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["review"], padding="max_length", truncation=True)

In [ ]:
dataset = load_dataset(dataset_name)

In [ ]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [ ]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))

In [ ]:
metric = evaluate.load("accuracy")

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
training_args = TrainingArguments(output_dir="test_trainer", eval_strategy="epoch")

In [ ]:
trainer = Trainer(
    model=model_classify,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()